In [1]:
import pandas as pd

# Nombres de los archivos en formato parquet
archivos_parquet = ['clean_df_ur.parquet', 'definitivo_df_ui.parquet', 'definitivo_df_sg.parquet']

# Cargar el primer archivo parquet
df_ur = pd.read_parquet('../DataSet/' + archivos_parquet[0])

In [2]:
# Cargar el segundo archivo parquet
df_ui = pd.read_parquet('../DataSet/' + archivos_parquet[1])


In [3]:

# Cargar el tercer archivo parquet
df_sg = pd.read_parquet('../DataSet/' + archivos_parquet[2])

In [4]:
df_ur

,user_id,item_id,recommend,posted_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0
...,...,...,...,...,...
59300,76561198312638244,70,True,2016,2
59301,76561198312638244,362890,True,2016,2
59302,LydiaMorley,273110,True,2016,2
59303,LydiaMorley,730,True,2016,2


In [5]:
df_ui


,user_id,item_id,item_name,playtime_forever,playtime_forever_log
0,76561197970982479,10,Counter-Strike,6.0,1.945910
1,76561197970982479,30,Day of Defeat,7.0,2.079442
2,76561197970982479,300,Day of Defeat: Source,4733.0,8.462526
3,76561197970982479,240,Counter-Strike: Source,1853.0,7.525101
4,76561197970982479,3830,Psychonauts,333.0,5.811141
...,...,...,...,...,...
3285241,76561198329548331,304930,Unturned,677.0,6.519147
3285242,76561198329548331,227940,Heroes & Generals,43.0,3.784190
3285243,76561198329548331,388490,One Way To Die: Steam Edition,3.0,1.386294
3285244,76561198329548331,521570,You Have 10 Seconds 2,4.0,1.609438


In [6]:
df_sg

,price,id,genres,developer,release_year
0,4.99,761140,"Action, Casual, Indie, Simulation, Strategy",Kotoshiro,2018
1,0.00,643980,"Free to Play, Indie, RPG, Strategy",Secret Level SRL,2018
2,0.00,670290,"Casual, Free to Play, Indie, Simulation, Sports",Poolians.com,2017
3,0.99,767400,"Action, Adventure, Casual",彼岸领域,2017
4,3.99,772540,"Action, Adventure, Simulation",Trickjump Games Ltd,2018
...,...,...,...,...,...
27630,1.99,745400,"Action, Adventure, Casual, Indie",Bidoniera Games,2018
27631,1.99,773640,"Casual, Indie, Simulation, Strategy","Nikita ""Ghost_RUS""",2018
27632,4.99,733530,"Casual, Indie, Strategy",Sacada,2018
27633,1.99,610660,"Indie, Racing, Simulation",Laush Dmitriy Sergeevich,2018


# Función 1

In [7]:
def developer(desarrollador):
    # Filtrar el DataFrame para incluir solo los juegos del desarrollador especificado
    df_dev = df_sg[df_sg['developer'] == desarrollador]

    # Agrupar por año de lanzamiento
    grouped = df_dev.groupby('release_year').agg(
        Cantidad_de_Items=('id', 'count'),  # Contar la cantidad de juegos por año
        Contenido_Free=('price', lambda x: (x == 0).sum())  # Contar juegos gratuitos
    )

    # Calcular el porcentaje de contenido gratuito
    grouped['Porcentaje_Contenido_Free'] = (grouped['Contenido_Free'] / grouped['Cantidad_de_Items']) * 100

    # Formatear el DataFrame para la presentación de resultados
    grouped = grouped.reset_index()
    grouped = grouped.rename(columns={'release_year': 'Año'})
    grouped['Porcentaje_Contenido_Free'] = grouped['Porcentaje_Contenido_Free'].apply(lambda x: f'{x:.2f}%')

    return grouped[['Año', 'Cantidad_de_Items', 'Porcentaje_Contenido_Free']]

In [8]:
# Ejemplo de cómo usar la función
resultado = developer("Kotoshiro")
print(resultado)

    Año  Cantidad_de_Items Porcentaje_Contenido_Free
0  2018                  1                     0.00%


# Función 2

In [9]:
def userdata(user_id):
    # Filtrar df_ui para el usuario especificado
    user_items = df_ui[df_ui['user_id'] == user_id]
    
    # Obtener los precios de los juegos que el usuario posee, utilizando df_sg
    precios_juegos = df_sg[df_sg['id'].isin(user_items['item_id'])]

    # Calcular el total de dinero gastado
    total_gastado = precios_juegos['price'].sum()

    # Calcular el porcentaje de recomendación
    user_reviews = df_ur[df_ur['user_id'] == user_id]
    if len(user_reviews) > 0:
        porcentaje_recomendacion = (user_reviews['recommend'].sum() / len(user_reviews)) * 100
    else:
        porcentaje_recomendacion = 0

    # Calcular la cantidad total de items
    cantidad_items = user_items['item_id'].nunique()

    # Preparar y retornar el resultado
    resultado = {
        "Usuario": user_id,
        "Dinero gastado": f"{total_gastado} USD",
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "cantidad de items": cantidad_items
    }

    return resultado


In [10]:
# Ejemplo de cómo usar la función
info_usuario = userdata(76561197970982479)
print(info_usuario)

{'Usuario': 76561197970982479, 'Dinero gastado': '0.0 USD', '% de recomendación': '0.00%', 'cantidad de items': 0}


# Función 3

In [11]:
def UserForGenre(genero):
    # Filtrar df_sg para juegos del género especificado
    juegos_genero = df_sg[df_sg['genres'].str.contains(genero)]

    # Unir df_ui con juegos_genero para obtener las horas jugadas en ese género por usuario
    horas_por_usuario = pd.merge(df_ui, juegos_genero, left_on='item_id', right_on='id')

    # Agrupar por usuario y sumar las horas jugadas totales
    total_horas_usuario = horas_por_usuario.groupby('user_id')['playtime_forever'].sum()

    # Encontrar el usuario con más horas jugadas
    usuario_max_horas = total_horas_usuario.idxmax()

    # Agrupar por año de lanzamiento y sumar las horas jugadas
    horas_por_año = horas_por_usuario.groupby('release_year')['playtime_forever'].sum().reset_index()
    horas_por_año = horas_por_año.rename(columns={'playtime_forever': 'Horas', 'release_year': 'Año'})

    # Convertir las horas acumuladas por año en una lista de diccionarios
    horas_lista = horas_por_año.to_dict('records')

    # Preparar y retornar el resultado
    resultado = {
        "Usuario con más horas jugadas para Género " + genero: usuario_max_horas,
        "Horas jugadas": horas_lista
    }

    return resultado



In [12]:
# Ejemplo de uso de la función
resultado = UserForGenre("Action")
print(resultado)


{'Usuario con más horas jugadas para Género Action': 'Sp3ctre', 'Horas jugadas': [{'Año': 1983, 'Horas': 3582.0}, {'Año': 1984, 'Horas': 384.0}, {'Año': 1988, 'Horas': 16243.0}, {'Año': 1989, 'Horas': 607.0}, {'Año': 1990, 'Horas': 18723.0}, {'Año': 1991, 'Horas': 2203.0}, {'Año': 1992, 'Horas': 1925.0}, {'Año': 1993, 'Horas': 23951.0}, {'Año': 1994, 'Horas': 120316.0}, {'Año': 1995, 'Horas': 217629.0}, {'Año': 1996, 'Horas': 70061.0}, {'Año': 1997, 'Horas': 363033.0}, {'Año': 1998, 'Horas': 2920063.0}, {'Año': 1999, 'Horas': 2726818.0}, {'Año': 2000, 'Horas': 18643971.0}, {'Año': 2001, 'Horas': 1337303.0}, {'Año': 2002, 'Horas': 740382.0}, {'Año': 2003, 'Horas': 11969297.0}, {'Año': 2004, 'Horas': 131222294.0}, {'Año': 2005, 'Horas': 12080963.0}, {'Año': 2006, 'Horas': 10899167.0}, {'Año': 2007, 'Horas': 17713042.0}, {'Año': 2008, 'Horas': 33960986.0}, {'Año': 2009, 'Horas': 165115925.0}, {'Año': 2010, 'Horas': 182664653.0}, {'Año': 2011, 'Horas': 335796534.0}, {'Año': 2012, 'Horas': 

# Función 4

In [13]:
def best_developer_year(año):
    # Filtrar df_ur para obtener solo las recomendaciones del año dado
    recomendaciones_año = df_ur[(df_ur['posted_year'] == año) & (df_ur['recommend'] == True)]

    # Unir df_ur filtrado con df_sg para obtener información del desarrollador
    recomendaciones_desarrollador = pd.merge(recomendaciones_año, df_sg, left_on='item_id', right_on='id')

    # Contar las recomendaciones por desarrollador
    conteo_recomendaciones = recomendaciones_desarrollador.groupby('developer')['user_id'].count().sort_values(ascending=False)

    # Obtener el top 3 de desarrolladores
    top_3_desarrolladores = conteo_recomendaciones.head(3).index.tolist()

    # Preparar y retornar el resultado en el formato deseado
    resultado = [{"Puesto " + str(i+1): dev} for i, dev in enumerate(top_3_desarrolladores)]

    return resultado




In [14]:
# Ejemplo de uso de la función
resultado = best_developer_year(2010)
print(resultado)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Tripwire Interactive'}, {'Puesto 3': 'Facepunch Studios'}]


# Función 5

In [15]:
def developer_reviews_analysis(desarrolladora):
    # Unir df_ur con df_sg para obtener reseñas de juegos del desarrollador especificado
    reseñas_desarrollador = pd.merge(df_ur, df_sg[df_sg['developer'] == desarrolladora], left_on='item_id', right_on='id')

    # Contar las reseñas positivas y negativas de 'sentiment_analysis'
    conteo_positivas = (reseñas_desarrollador['sentiment_analysis'] == 2).sum()
    conteo_negativas = (reseñas_desarrollador['sentiment_analysis'] == 0).sum()

    # Preparar y retornar el resultado
    resultado = {
        desarrolladora: {
            'Negative': conteo_negativas,
            'Positive': conteo_positivas
        }
    }

    return resultado

In [16]:
# Ejemplo de uso de la función
resultado = developer_reviews_analysis("Valve")
print(resultado)

{'Valve': {'Negative': 1609, 'Positive': 5384}}
